## Imports

In [1]:
import os
import pandas as pd
import json
import re

In [2]:
%run ../utils/__init__.py
%run ../metrics/__init__.py

## Functions

In [13]:
classification = False

In [14]:
BASE_FOLDER = os.path.join(WORKSPACE_DIR,
                           'classification' if classification else 'report_generation')
RESULTS_FOLDER = os.path.join(BASE_FOLDER, 'results')

In [15]:
def get_suffix(filename):
    match = re.search('.*metrics-(?P<suffix>\w*)\.json', filename)
    if match is None:
        suffix = ''
    else:
        suffix = match.group('suffix')
    return suffix

In [16]:
KEY_COLS = ['run_name', 'dataset_type']
if not classification:
    KEY_COLS.append('free')
KEY_COLS

['run_name', 'dataset_type', 'free']

In [32]:
METRIC_TYPES = [
    'chexpert',
    'grad-cam',
    'mirqi',
]

In [33]:
def load_results():
    results_by_metric_type = {}

    for run_name in os.listdir(RESULTS_FOLDER):
        if run_name == 'debug':
            continue

        folder = os.path.join(RESULTS_FOLDER, run_name)
        for filename in os.listdir(folder):
            filepath = os.path.join(folder, filename)
            if not os.path.isfile(filepath) or not filename.endswith('json'):
                continue

            metric_type = next(
                (met for met in METRIC_TYPES if met in filename),
                'base', # Default if no specific metric_type is found
            )

            with open(filepath, 'r') as f:
                results_dict = json.load(f)
            
            results_df = pd.DataFrame.from_dict(results_dict, orient='index')
            results_df.reset_index(inplace=True)
            results_df.rename(columns={'index': 'dataset_type'}, inplace=True)
            results_df['run_name'] = run_name
            if not classification:
                results_df['free'] = get_suffix(filename)           
            
            if metric_type not in results_by_metric_type:
                results_by_metric_type[metric_type] = results_df
            else:
                prev = results_by_metric_type[metric_type]
                results_by_metric_type[metric_type] = prev.append(results_df, ignore_index=True)

    df = None
    cols_in_order = list(KEY_COLS)
    for results in results_by_metric_type.values():
        cols_in_order += [col for col in results.columns if col not in cols_in_order]
        
        if df is None:
            df = results
        else:
            df = df.merge(results, on=KEY_COLS, how='outer')
                
    return df[cols_in_order], results_by_metric_type

In [38]:
def filter_results(dataset_type=None, metrics=None,
                   metrics_contain=None, free=None, contains=None,
                   drop=None, drop_na_rows=False):
    df = RESULTS_DF
    
    if dataset_type:
        if isinstance(dataset_type, str):
            df = df[df['dataset_type'] == dataset_type]
        elif isinstance(dataset_type, (list, tuple)):
            dataset_type = set(dataset_type)
            df = df[df['dataset_type'].isin(dataset_type)]
    
    if free is not None:
        free_str = 'free' if free else 'notfree'
        df = df.loc[df['free'] == free_str]
    
    if contains:
        filter_contains = lambda d, s: d.loc[d['run_name'].str.contains(s)]
        if isinstance(contains, (list, tuple)):
            for c in contains:
                df = filter_contains(df, c)
        elif isinstance(contains, str):
            df = filter_contains(df, contains)
    
    if drop:
        df = df.loc[~df['run_name'].str.contains(drop)]
        
    if metrics_contain:
        columns = KEY_COLS + [c for c in df.columns if metrics_contain in c]
        df = df[columns]
    elif metrics:
        columns = KEY_COLS + metrics
        df = df[columns]
    
    if drop_na_rows:
        df.dropna(axis=0, how='any', inplace=True)

    return df

## Load results

In [62]:
RESULTS_DF, debug = load_results()
RESULTS_DF.head()

,run_name,dataset_type,free,MIRQI-r,MIRQI-p,MIRQI-f,MIRQI-v2-r,MIRQI-v2-p,MIRQI-v2-sp,MIRQI-v2-np,...,bleu1,bleu2,bleu3,bleu4,bleu,rougeL,ciderD,distinct_words,distinct_sentences,word_acc
0,0915_174026_dummy-common-sentences-100,test,notfree,0.419810,0.427106,0.418189,0.309874,0.325798,0.826896,0.788923,...,0.403291,0.249472,0.165749,0.116085,0.233649,0.300978,0.273964,109.0,325.0,0.040873
1,0915_174026_dummy-common-sentences-100,train,notfree,0.441749,0.447833,0.439091,0.333208,0.349260,0.833635,0.788656,...,0.391287,0.241733,0.159835,0.110573,0.225857,0.297524,0.226741,109.0,1028.0,0.038364
2,0915_174026_dummy-common-sentences-100,val,notfree,0.445693,0.453462,0.444027,0.340213,0.357523,0.825500,0.783091,...,0.393880,0.241507,0.160103,0.111258,0.226687,0.295175,0.265806,109.0,339.0,0.037777
3,0915_174026_dummy-common-sentences-100,test,free,0.417426,0.415991,0.413998,0.310505,0.315160,0.813117,0.784744,...,0.346689,0.210850,0.138440,0.095769,0.197937,0.280916,0.165551,109.0,222.0,NaN
4,0915_174026_dummy-common-sentences-100,train,free,0.440705,0.444578,0.438526,0.343075,0.354044,0.780014,0.746212,...,0.340114,0.207458,0.137165,0.095464,0.195050,0.276580,0.152633,109.0,296.0,NaN


## Report generation

In [63]:
NLP_METRICS = ['bleu1', 'bleu2', 'bleu3', 'bleu4', 'bleu', 'rougeL', 'ciderD']
CHEXPERT_METRICS = ['acc', 'roc_auc'] #  'recall', 'prec', 'f1', 'roc_auc']
MIRQI_METRICS = [c for c in RESULTS_DF.columns if 'MIRQI' in c]

In [64]:
filter_results(contains='0918_001139')

,run_name,dataset_type,free,MIRQI-r,MIRQI-p,MIRQI-f,MIRQI-v2-r,MIRQI-v2-p,MIRQI-v2-sp,MIRQI-v2-np,...,bleu1,bleu2,bleu3,bleu4,bleu,rougeL,ciderD,distinct_words,distinct_sentences,word_acc
534,0918_001139_h-lstm_lr0.0001_densenet-121_size256,train,free,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.345567,0.221134,0.155748,0.114807,0.209314,0.340792,0.193935,23.0,211.0,NaN
535,0918_001139_h-lstm_lr0.0001_densenet-121_size256,val,free,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.377572,0.239016,0.165878,0.120773,0.225810,0.339514,0.177418,22.0,69.0,NaN
536,0918_001139_h-lstm_lr0.0001_densenet-121_size256,test,free,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.386300,0.247744,0.172090,0.125292,0.232856,0.347354,0.176782,22.0,83.0,NaN
537,0918_001139_h-lstm_lr0.0001_densenet-121_size256,test-normal,free,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.391145,0.273677,0.203930,0.157574,0.256582,0.406953,0.236378,NaN,NaN,NaN
538,0918_001139_h-lstm_lr0.0001_densenet-121_size256,test-abnormal,free,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.383784,0.233424,0.154277,0.107048,0.219633,0.314734,0.153749,NaN,NaN,NaN
539,0918_001139_h-lstm_lr0.0001_densenet-121_size256,val-normal,free,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.367432,0.249210,0.183503,0.140432,0.235144,0.381029,0.236243,NaN,NaN,NaN
540,0918_001139_h-lstm_lr0.0001_densenet-121_size256,val-abnormal,free,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.383420,0.231127,0.152844,0.106324,0.218429,0.311280,0.148821,NaN,NaN,NaN
541,0918_001139_h-lstm_lr0.0001_densenet-121_size256,train-normal,free,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.349597,0.238317,0.175787,0.134498,0.224550,0.388275,0.279540,NaN,NaN,NaN
542,0918_001139_h-lstm_lr0.0001_densenet-121_size256,train-abnormal,free,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.343129,0.210277,0.143041,0.102298,0.199686,0.309795,0.139781,NaN,NaN,NaN
543,0918_001139_h-lstm_lr0.0001_densenet-121_size256,train,notfree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.410230,0.260711,0.182669,0.134456,0.247016,0.355782,0.215192,23.0,377.0,0.059400


In [39]:
filter_results(dataset_type='test',
               metrics=MIRQI_METRICS,
               free=True, drop_na_rows=True)

,run_name,dataset_type,free,MIRQI-r,MIRQI-p,MIRQI-f,MIRQI-v2-r,MIRQI-v2-p,MIRQI-v2-sp,MIRQI-v2-np,MIRQI-v2-f,MIRQI-v2-attr-p,MIRQI-v2-attr-r
3,0915_174026_dummy-common-sentences-100,test,free,0.417426,0.415991,0.413998,0.310505,0.315160,0.813117,0.784744,0.311550,0.305851,0.305851
15,0915_173524_dummy-common-sentences-10,test,free,0.457447,0.449426,0.452115,0.356383,0.356383,0.813829,0.773725,0.356383,0.356383,0.356383
21,0915_172709_dummy-constant,test,free,0.478457,0.457842,0.465502,0.356383,0.356383,0.964095,0.861018,0.356383,0.356383,0.356383
28,0918_144929_lstm-att-v2_lr0.0001_densenet-121_...,test,free,0.468323,0.467369,0.461983,0.360217,0.370789,0.896238,0.840001,0.360826,0.288435,0.284874
35,0919_144203_h-lstm-att-v2_lr0.0001_resnet-50_s...,test,free,0.467021,0.460237,0.462798,0.356383,0.356383,0.893617,0.859697,0.356383,0.356383,0.356383
41,0918_125940_lstm-v2_lr0.0001_densenet-121_size256,test,free,0.448021,0.441593,0.442201,0.337433,0.341755,0.874132,0.823266,0.338564,0.321698,0.321077
45,0918_190428_h-lstm-att-v2_lr0.0001_densenet-12...,test,free,0.466755,0.461181,0.463214,0.356383,0.356383,0.892287,0.864418,0.356383,0.356383,0.356383
51,0919_054554_lstm-v2_lr0.0001_mobilenet_size256,test,free,0.444598,0.441769,0.441611,0.344415,0.345745,0.807136,0.785970,0.344858,0.329122,0.328236
58,0915_173609_dummy-random,test,free,0.431931,0.430070,0.423154,0.344548,0.343018,0.726773,0.726130,0.336981,0.175747,0.174897
64,0918_164827_h-lstm-v2_lr0.0001_densenet-121_si...,test,free,0.467021,0.459003,0.461981,0.356383,0.356383,0.893617,0.853527,0.356383,0.356383,0.356383


In [40]:
metrics = [
    col
    for col in RESULTS_DF.columns
    # if col.startswith('recall')
    # if col == 'recall-No Finding'
    # if 'No Finding' in col
]
cols = KEY_COLS + metrics
cols
# RESULTS_DF[cols].sort_index()

['run_name',
 'dataset_type',
 'free',
 'run_name',
 'dataset_type',
 'free',
 'MIRQI-r',
 'MIRQI-p',
 'MIRQI-f',
 'MIRQI-v2-r',
 'MIRQI-v2-p',
 'MIRQI-v2-sp',
 'MIRQI-v2-np',
 'MIRQI-v2-f',
 'MIRQI-v2-attr-p',
 'MIRQI-v2-attr-r',
 'acc',
 'acc-No Finding',
 'acc-Enlarged Cardiomediastinum',
 'acc-Cardiomegaly',
 'acc-Lung Lesion',
 'acc-Lung Opacity',
 'acc-Edema',
 'acc-Consolidation',
 'acc-Pneumonia',
 'acc-Atelectasis',
 'acc-Pneumothorax',
 'acc-Pleural Effusion',
 'acc-Pleural Other',
 'acc-Fracture',
 'acc-Support Devices',
 'prec',
 'prec-No Finding',
 'prec-Enlarged Cardiomediastinum',
 'prec-Cardiomegaly',
 'prec-Lung Lesion',
 'prec-Lung Opacity',
 'prec-Edema',
 'prec-Consolidation',
 'prec-Pneumonia',
 'prec-Atelectasis',
 'prec-Pneumothorax',
 'prec-Pleural Effusion',
 'prec-Pleural Other',
 'prec-Fracture',
 'prec-Support Devices',
 'recall',
 'recall-No Finding',
 'recall-Enlarged Cardiomediastinum',
 'recall-Cardiomegaly',
 'recall-Lung Lesion',
 'recall-Lung Opacity'

In [41]:
replace_strs = [
    (r'^\d{4}_\d{6}_', ''),
    ('most-similar-image', '1nn'),
    ('_lr[\d\.]+', ''),
    ('_size256', ''),
    (r'_\d{4}_\d{6}_.*', ''),
    ('dummy-', ''),
    ('common', 'top'),
    ('-v2', ''),
    (r'top-(\w)\w+-(\d+)', r'top-\1-\2'),
    ('_densenet-121', ''),
]

def rename_runs(run_name):
    s = run_name
    for target, replace_with in replace_strs:
        s = re.sub(target, replace_with, s)
    return s

In [56]:
columns = ['bleu', 'rougeL', 'ciderD'] + CHEXPERT_METRICS + MIRQI_METRICS

In [57]:
df = filter_results(dataset_type='test',
                    free=True,
                    metrics=columns,
                    contains='(?=_lstm-att-v2.*densenet|_lstm-v2.*densenet|dummy)',
                    drop='0915_173951|0915_174222|0916_104739',
                    drop_na_rows=True,
                   )

In [51]:
df

,run_name,dataset_type,free,bleu1,bleu2,bleu3,bleu4,bleu,rougeL,ciderD,...,MIRQI-r,MIRQI-p,MIRQI-f,MIRQI-v2-r,MIRQI-v2-p,MIRQI-v2-sp,MIRQI-v2-np,MIRQI-v2-f,MIRQI-v2-attr-p,MIRQI-v2-attr-r
3,0915_174026_dummy-common-sentences-100,test,free,0.346689,0.210850,0.138440,9.576881e-02,0.197937,0.280916,0.165551,...,0.417426,0.415991,0.413998,0.310505,0.315160,0.813117,0.784744,0.311550,0.305851,0.305851
9,0915_172915_dummy-common-words-10,test,free,0.312055,0.107749,0.024166,8.550974e-07,0.110993,0.243604,0.042768,...,0.356383,0.356383,0.356383,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,0915_173524_dummy-common-sentences-10,test,free,0.275183,0.170479,0.112833,7.838136e-02,0.159219,0.294405,0.151610,...,0.457447,0.449426,0.452115,0.356383,0.356383,0.813829,0.773725,0.356383,0.356383,0.356383
21,0915_172709_dummy-constant,test,free,0.454623,0.311525,0.223393,1.653650e-01,0.288726,0.356681,0.292646,...,0.478457,0.457842,0.465502,0.356383,0.356383,0.964095,0.861018,0.356383,0.356383,0.356383
28,0918_144929_lstm-att-v2_lr0.0001_densenet-121_...,test,free,0.360850,0.226277,0.152071,1.064681e-01,0.211416,0.313536,0.186868,...,0.468323,0.467369,0.461983,0.360217,0.370789,0.896238,0.840001,0.360826,0.288435,0.284874
41,0918_125940_lstm-v2_lr0.0001_densenet-121_size256,test,free,0.379042,0.239385,0.163692,1.170487e-01,0.224792,0.337529,0.284303,...,0.448021,0.441593,0.442201,0.337433,0.341755,0.874132,0.823266,0.338564,0.321698,0.321077
58,0915_173609_dummy-random,test,free,0.362480,0.196943,0.117449,7.459662e-02,0.187867,0.263606,0.111804,...,0.431931,0.430070,0.423154,0.344548,0.343018,0.726773,0.726130,0.336981,0.175747,0.174897
107,0915_173754_dummy-common-sentences-50,test,free,0.332812,0.205903,0.137483,9.688422e-02,0.193271,0.282765,0.188344,...,0.421740,0.424866,0.420113,0.318429,0.327128,0.802019,0.777257,0.320464,0.313830,0.313830
108,0916_104837_dummy-most-similar-image_0717_1202...,test,free,0.382749,0.220256,0.142339,9.958416e-02,0.211232,0.288225,0.230329,...,0.435919,0.444452,0.433369,0.347119,0.355474,0.754784,0.753767,0.345257,0.186351,0.185880
115,0915_173307_dummy-common-words-100,test,free,0.348717,0.089660,0.016369,6.481232e-07,0.113687,0.206325,0.071575,...,0.224643,0.209504,0.200727,0.198094,0.160554,0.207445,0.328014,0.168722,0.059774,0.059563


In [59]:
shorten_cols = lambda s: s.replace('MIRQI-v2', 'v2')

In [61]:
print(df.set_index('run_name').rename(
    index=rename_runs,
    columns=shorten_cols,
).sort_index().to_latex(
    columns=[shorten_cols(c) for c in columns],
    float_format='%.3f',
    column_format='l' + 'c' * len(columns),
))

\begin{tabular}{lccccccccccccccc}
\toprule
{} &  bleu &  rougeL &  ciderD &   acc &  roc\_auc &  MIRQI-r &  MIRQI-p &  MIRQI-f &  v2-r &  v2-p &  v2-sp &  v2-np &  v2-f &  v2-attr-p &  v2-attr-r \\
run\_name  &       &         &         &       &          &          &          &          &       &       &        &        &       &            &            \\
\midrule
1nn       & 0.211 &   0.288 &   0.230 & 0.903 &    0.518 &    0.436 &    0.444 &    0.433 & 0.347 & 0.355 &  0.755 &  0.754 & 0.345 &      0.186 &      0.186 \\
constant  & 0.289 &   0.357 &   0.293 & 0.915 &    0.500 &    0.478 &    0.458 &    0.466 & 0.356 & 0.356 &  0.964 &  0.861 & 0.356 &      0.356 &      0.356 \\
lstm      & 0.225 &   0.338 &   0.284 & 0.912 &    0.505 &    0.448 &    0.442 &    0.442 & 0.337 & 0.342 &  0.874 &  0.823 & 0.339 &      0.322 &      0.321 \\
lstm-att  & 0.211 &   0.314 &   0.187 & 0.918 &    0.508 &    0.468 &    0.467 &    0.462 & 0.360 & 0.371 &  0.896 &  0.840 & 0.361 &      0.288 &  

## Classification

In [64]:
# contains = 'covid-x'
# contains = 'cxr14'
contains = 'e0'
# contains = '0717_120222_covid-x_densenet-121_lr1e-06_os_aug-covid'
# contains = '0717_101812_covid-x_densenet-121_lr1e-06_os-max2_aug-covid'
run_name = '0717_120222_covid-x_densenet-121_lr1e-06_os_aug-covid' # WINNER

contains = '0717_101812_covid-x_densenet-121_lr1e-06_os-max2_aug-covid'
contains = 'covid-uc'

In [12]:
filter_results(dataset_type='test-bbox', metrics_contain='iou')

,run_name,dataset_type,iou-Pneumonia,iou-Atelectasis,iou-Cardiomegaly,iou-Effusion,iou-Infiltration,iou-Mass,iou-Nodule,iou-Pneumothorax,iou-Consolidation,iou-Edema,iou-Emphysema,iou-Fibrosis,iou-Pleural_Thickening,iou-Hernia
172,0715_232631_cxr14_densenet-121_lr1e-06_os-max4...,test-bbox,0.104788,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
173,1027_144914_cxr14_densenet-121_lr1e-05_aug_nor...,test-bbox,NaN,NaN,NaN,0.089691,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
174,0722_121738_cxr14_densenet-121_lr0.0001_aug-0_...,test-bbox,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
175,0714_232500_cxr14_densenet-121_lr1e-06,test-bbox,0.082394,0.034604,0.183337,0.091254,0.097544,0.053841,0.004207,0.054527,0.0,0.0,0.0,0.0,0.0,0.0
176,0715_232418_cxr14_densenet-121_lr1e-06_os-max4...,test-bbox,0.102067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177,0721_125845_cxr14_densenet-121_lr1e-05_aug-0_b...,test-bbox,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178,0721_124730_cxr14_densenet-121_lr1e-05_bce_Car...,test-bbox,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179,0716_133211_cxr14_densenet-121_lr1e-06_us_aug-...,test-bbox,0.048599,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
180,0714_232518_cxr14_densenet-121_lr1e-06,test-bbox,0.102803,0.027257,0.145093,0.085544,0.037971,0.033063,0.005916,0.057814,0.0,0.0,0.0,0.0,0.0,0.0
181,0720_213359_cxr14_densenet-121_lr1e-06_aug-0_b...,test-bbox,NaN,NaN,0.197302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
results_df_test.sort_values(['acc', 'recall_covid', 'prec_covid'],
                            ascending=False).dropna(axis=1, how='all')

""


In [36]:
print_columns = [
    'acc',
    'prec_covid','prec_pneumonia','prec_normal',
    'recall_covid','recall_pneumonia', 'recall_normal',
    'spec_covid','spec_pneumonia', 'spec_normal'
#     'acc_Pneumonia', 'prec_Pneumonia', 'recall_Pneumonia'
]
# print_columns = [c for c in results_df_test.columns if c != 'loss' and 'cm' not in c]

In [37]:
print((results_df_test*100).to_latex(columns=print_columns,
                               float_format='%.2f',
                              ))

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} &   acc &  prec\_covid &  prec\_pneumonia &  prec\_normal &  recall\_covid &  recall\_pneumonia &  recall\_normal &  spec\_covid &  spec\_pneumonia &  spec\_normal \\
\midrule
0717\_101812\_covid-x\_densenet-121\_lr1e-06\_os-max... & 93.73 &       93.94 &           91.12 &        95.53 &         93.00 &             93.27 &          94.12 &       99.59 &           94.52 &        94.38 \\
\bottomrule
\end{tabular}



## Report-generation: results at different report lengths

In [32]:
vals_words = [20, 25, 27, 33, 44, None]
vals_sents = [3, 4, 5, 6, None]

In [40]:
max_words = vals_words[0]
suffix = f'max-words-{max_words}' if max_words else ''
all_results = load_results(suffix)
results_df_test = create_results_df(all_results, 'test')
results_df_test

,loss,word_acc,bleu1,bleu2,bleu3,bleu4,bleu,rougeL,ciderD
0717_041434_lstm_lr0.0001_densenet-121,5.579509,0.149129,0.478934,0.325275,0.256634,0.218300,0.319786,0.395486,1.354237
0716_211601_lstm-att_lr0.0001_densenet-121,7.382415,0.068293,0.442359,0.282490,0.199632,0.150046,0.268632,0.358223,0.775244
0717_015057_h-lstm_lr0.0001_densenet-121,4.548621,0.162518,0.328184,0.195510,0.126410,0.082401,0.183126,0.334782,0.256378
0716_234501_h-lstm-att_lr0.0001_densenet-121,5.157984,0.132504,0.298223,0.179060,0.115951,0.075574,0.167202,0.315891,0.101786


## DEBUG

In [48]:
%run -n ../eval_report_generation_chexpert_labeler.py

ModuleNotFoundError: No module named 'medai.dataset'

In [29]:
import pandas as pd
import os

In [43]:
fpath = '/mnt/workspace/iu-x-ray/dataset/reports/reports_with_chexpert_labels.csv'
df = pd.read_csv(fpath, index_col=0)
df.replace((-1, -2), 0, inplace=True)
df.head()

,Reports,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Lesion,Lung Opacity,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,filename
0,the cardiac silhouette and mediastinum size ar...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.xml
1,the cardiomediastinal silhouette is within nor...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.xml
2,both lungs are clear and expanded . heart and ...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.xml
3,there is xxxx increased opacity within the rig...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1000.xml
4,interstitial markings are diffusely prominent ...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1001.xml


In [41]:
rename_cols = { 'Reports': 'ground_truth' }
for label in CHEXPERT_LABELS:
    rename_cols[label] = f'{label}-gt'
rename_cols

{'Reports': 'ground_truth',
 'No Finding': 'No Finding-gt',
 'Enlarged Cardiomediastinum': 'Enlarged Cardiomediastinum-gt',
 'Cardiomegaly': 'Cardiomegaly-gt',
 'Lung Lesion': 'Lung Lesion-gt',
 'Lung Opacity': 'Lung Opacity-gt',
 'Edema': 'Edema-gt',
 'Consolidation': 'Consolidation-gt',
 'Pneumonia': 'Pneumonia-gt',
 'Atelectasis': 'Atelectasis-gt',
 'Pneumothorax': 'Pneumothorax-gt',
 'Pleural Effusion': 'Pleural Effusion-gt',
 'Pleural Other': 'Pleural Other-gt',
 'Fracture': 'Fracture-gt',
 'Support Devices': 'Support Devices-gt'}

In [42]:
df.rename(columns=rename_cols, inplace=True)
df.head()

,ground_truth,No Finding-gt,Enlarged Cardiomediastinum-gt,Cardiomegaly-gt,Lung Lesion-gt,Lung Opacity-gt,Edema-gt,Consolidation-gt,Pneumonia-gt,Atelectasis-gt,Pneumothorax-gt,Pleural Effusion-gt,Pleural Other-gt,Fracture-gt,Support Devices-gt,filename
0,the cardiac silhouette and mediastinum size ar...,1.0,0.0,0.0,-2.0,-2.0,0.0,0.0,-2.0,-2.0,0.0,0.0,-2.0,-2.0,-2.0,1.xml
1,the cardiomediastinal silhouette is within nor...,-2.0,0.0,-2.0,-2.0,0.0,-2.0,-2.0,-2.0,-2.0,0.0,0.0,-2.0,-2.0,-2.0,10.xml
2,both lungs are clear and expanded . heart and ...,1.0,0.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,100.xml
3,there is xxxx increased opacity within the rig...,-2.0,-2.0,0.0,-1.0,1.0,-2.0,-1.0,-2.0,-1.0,0.0,0.0,-2.0,-2.0,-2.0,1000.xml
4,interstitial markings are diffusely prominent ...,-2.0,-2.0,0.0,-2.0,1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,1001.xml


In [46]:
run_name = '0919_011224_h-lstm-v2_lr0.0001_precnn_size256'
debug = False
free = False

In [47]:
results_folder = get_results_folder(run_name, classification=False, debug=debug)

# Output file at the end of this process
suffix = 'free' if free else 'notfree'
labeled_output_path = os.path.join(results_folder, f'outputs-labeled-{suffix}.csv')

model_output_path = os.path.join(results_folder, f'outputs-{suffix}.csv')

if not os.path.isfile(model_output_path):
    print('Need to compute outputs for run first: ', model_output_path)

# Read outputs
df2 = pd.read_csv(model_output_path)
df2.head()

,filename,epoch,dataset_type,ground_truth,generated
0,2089.xml,1,train,no active disease .,the heart is normal in size . the lungs are cl...
1,2089.xml,1,train,no active disease .,the heart is normal in size . the lungs are cl...
2,2435.xml,1,train,no active disease .,the heart is normal in size . the lungs are cl...
3,2435.xml,1,train,no active disease .,the heart is normal in size . the lungs are cl...
4,2460.xml,1,train,no active disease .,the heart is normal in size . the lungs are cl...


In [58]:
%run -n ../eval_report_generation_chexpert_labeler.py

In [56]:
labels = _load_gt_labels(df2)
labels.shape

(7426, 14)

In [57]:
labels

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [26]:
_save_cached_gt_labels(df)

KeyError: "None of [Index(['ground_truth', 'No Finding-gt', 'Enlarged Cardiomediastinum-gt',\n       'Cardiomegaly-gt', 'Lung Lesion-gt', 'Lung Opacity-gt', 'Edema-gt',\n       'Consolidation-gt', 'Pneumonia-gt', 'Atelectasis-gt', 'Pneumothorax-gt',\n       'Pleural Effusion-gt', 'Pleural Other-gt', 'Fracture-gt',\n       'Support Devices-gt'],\n      dtype='object')] are in the [columns]"